### **Ejercicio:** Análisis de una Competencia de Kaggle

**Contexto:**

Tienes un conjunto de datos en formato JSON que contiene información sobre una competencia de Kaggle. Los datos incluyen el ID del participante, el número de intentos, el tiempo invertido en la competencia y las puntuaciones obtenidas en cada intento. Algunos participantes tienen datos incompletos o valores nulos, que deberás manejar antes de realizar el análisis.

**Objetivos:**
- Cargar y explorar un archivo JSON.
- Identificar y manejar datos nulos.
- Calcular estadísticas descriptivas (promedios, máximos, mínimos, etc.) para el número de intentos, el tiempo invertido y las puntuaciones.
- Aplicar condicionales y operadores matemáticos para evaluar el rendimiento de los participantes (e.g., calcular el promedio de puntuaciones por intento).
- Guardar el conjunto de datos limpio en un nuevo archivo JSON.

In [12]:
import pandas as pd
import json

# Ruta del archivo JSON de entrada
file_path = r"C:\Users\Andy\Documents\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\competencia_kaggle.json"

# 1. Cargar el archivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)

# Convertir los datos JSON a un DataFrame
df = pd.json_normalize(data, 'puntuaciones', ['id_participante', 'nombre', 'intentos', 'tiempo_total_horas'], record_prefix='puntuacion_')

# 2. Convertir columnas a valores numéricos para análisis
df['intentos'] = pd.to_numeric(df['intentos'], errors='coerce')
df['tiempo_total_horas'] = pd.to_numeric(df['tiempo_total_horas'], errors='coerce')

# 3. Manejo de valores nulos en 'puntuacion_0' y 'tiempo_total_horas'
# Rellenar valores nulos en 'puntuacion_0' con el promedio de puntuaciones del participante
df['puntuacion_0'] = df.groupby('id_participante')['puntuacion_0'].transform(lambda x: x.fillna(x.mean()))
# Rellenar valores nulos en 'tiempo_total_horas' con el promedio general de tiempo total
df['tiempo_total_horas'] = df['tiempo_total_horas'].fillna(df['tiempo_total_horas'].mean())

# 4. Estadísticas descriptivas
stats_intentos = df['intentos'].describe()
stats_tiempo = df['tiempo_total_horas'].describe()
stats_puntuaciones = df['puntuacion_0'].describe()

# 5. Calcular el promedio de puntuación por intento para cada participante
df['promedio_por_intento'] = df.groupby('id_participante')['puntuacion_0'].transform('mean')

# 6. Resumen del rendimiento por participante
rendimiento_participantes = df.groupby(['id_participante', 'nombre']).agg(
    intentos=('intentos', 'first'),
    tiempo_total_horas=('tiempo_total_horas', 'first'),
    promedio_puntuacion=('promedio_por_intento', 'mean')
).reset_index()

# 7. Guardar el DataFrame limpio y resumido en un nuevo archivo JSON
output_path = r"C:\Users\Andy\Documents\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\competencia_kaggle_limpio.json"
rendimiento_participantes.to_json(output_path, orient='records', lines=True)

print("Estadísticas descriptivas de intentos:\n", stats_intentos)
print("Estadísticas descriptivas de tiempo total:\n", stats_tiempo)
print("Estadísticas descriptivas de puntuaciones:\n", stats_puntuaciones)


Estadísticas descriptivas de intentos:
 count    20.000000
mean      4.500000
std       1.357242
min       2.000000
25%       3.750000
50%       5.000000
75%       6.000000
max       6.000000
Name: intentos, dtype: float64
Estadísticas descriptivas de tiempo total:
 count    20.000000
mean     11.411765
std       2.636960
min       5.000000
25%      10.000000
50%      12.000000
75%      14.000000
max      14.000000
Name: tiempo_total_horas, dtype: float64
Estadísticas descriptivas de puntuaciones:
 count    20.000000
mean     80.837500
std       9.581807
min      60.000000
25%      77.250000
50%      81.625000
75%      86.312500
max      95.000000
Name: puntuacion_0, dtype: float64


### Algunos Análisis Adicionales

In [15]:

# Análisis adicional: relación entre intentos y puntuaciones promedio
promedio_intentos = df['intentos'].mean()
promedio_puntuacion = df['promedio_puntuacion'].mean()
correlacion_intentos_puntuacion = df[['intentos', 'promedio_puntuacion']].corr().iloc[0, 1]

# Mostrar resultados
print(f"\nPromedio de intentos por participante: {promedio_intentos:.2f}")
print(f"Promedio de puntuación por intento: {promedio_puntuacion:.2f}")
print(f"Correlación entre número de intentos y puntuación promedio: {correlacion_intentos_puntuacion:.2f}")



Promedio de intentos por participante: 4.00
Promedio de puntuación por intento: 78.05
Correlación entre número de intentos y puntuación promedio: 0.73


### Resumen del Análisis de Datos

- **Participantes e Intentos**: 
   - El conjunto de datos incluye múltiples participantes con un rango de intentos entre 2 y 6, promediando **4.5 intentos** por participante.
   - La distribución de intentos sugiere que la mayoría de los participantes realizó varios intentos para optimizar sus puntuaciones.

- **Distribución de Tiempo Total Invertido**: 
   - Los participantes invirtieron entre **5 y 14 horas** en la competencia, con un promedio de **11.4 horas**. 
   - La desviación estándar indica una variabilidad significativa en el tiempo invertido, posiblemente reflejando diferentes estrategias de compromiso.

- **Puntuación Promedio**: 
   - Las puntuaciones por intento varían entre **60 y 95 puntos**, con una media de **80.8 puntos**. 
   - La moderada desviación estándar refleja un rendimiento consistente entre los participantes.

- **Análisis de Correlación**: 
   - La correlación entre el número de intentos y la puntuación promedio es positiva pero moderada, lo que sugiere que aunque más intentos pueden mejorar el rendimiento, otros factores como habilidad técnica y estrategia juegan un papel importante.

